# load data

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")
diamonds = sns.load_dataset("diamonds")
diamonds.head()

In [ ]:
"""import data"""
from sklearn.model_selection import train_test_split
# Extract feature and target arrays
# X is all the other features and y is the price
X, y = diamonds.drop('price', axis=1), diamonds[['price']]

# Extract text features
# for the features that are not np.number, convert them to type 'category'. (eg. color column is 'blue', 'green' ...)
# (for xgboost, category datatype can be self transformed and recognised by the model, no need to manually do one hot)
cats = X.select_dtypes(exclude=np.number).columns.tolist()
for col in cats:
    X[col] = X[col].astype('category')

In [ ]:
"""train test split"""
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# sklearn API

In [ ]:
from xgboost import XGBClassifier
# read data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2)
# create model instance
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)

# XGboost training with native API, using regression as example

In [ ]:
"""convert the data to XGBoost specified DMatrix"""
import xgboost as xgb

dtrain_reg = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_reg = xgb.DMatrix(X_test, y_test, enable_categorical=True)

In [ ]:
"""specify hyper params in dict"""
# Define hyperparameters
params = {"objective": "reg:squarederror", "tree_method": "gpu_hist"}

n = 100
model = xgb.train(
    params=params,
    dtrain=dtrain_reg,
    num_boost_round=n,
)

# testing

result is an np_array

In [ ]:
preds = model.predict(dtest_reg)

# Other training objectives

## 1. classification

1. Just need to change the objective funciton
   The two most popular classification objectives are:
  > binary:logistic - binary classification (the target contains only two classes, i.e., cat or dog)   
  > multi:softprob - multi-class classification (more than two classes in the target, i.e., apple/orange/banana)

In [ ]:
"""multi-class (binary class is similar)"""

'''data'''
from sklearn.preprocessing import OrdinalEncoder
X, y = diamonds.drop("cut", axis=1), diamonds[['cut']]
# Encode y to numeric
# XGBoost only accepts number in tartget
y_encoded = OrdinalEncoder().fit_transform(y)
# Extract text features
cats = X.select_dtypes(exclude=np.number).columns.tolist()
# Convert to pd.Categorical
for col in cats:
    X[col] = X[col].astype('category')
# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, random_state=1, stratify=y_encoded)
# Create classification matrices
dtrain_clf = xgb.DMatrix(X_train, y_train, enable_categorical=True)
dtest_clf = xgb.DMatrix(X_test, y_test, enable_categorical=True)

'''train'''
params = {"objective": "multi:softprob", "tree_method": "gpu_hist", "num_class": 5}
n = 1000
results = xgb.cv(
    params, dtrain_clf,
    num_boost_round=n,
    nfold=5,
    metrics=["mlogloss", "auc", "merror"],
)

# multiclass classification

just that Input y should be of shape (n_samples, n_classes) with each column having a value of 0 or 1 to specify whether the sample is labeled as positive for respective class.